<a href="https://colab.research.google.com/github/HelenNunez/BALG_Project3/blob/aranza1/Training_Clustering_Test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install tensorflow

In [3]:
pip install --upgrade tensorflow-datasets

     |████████████████████████████████| 3.4MB 2.9MB/s 
  Found existing installation: tensorflow-datasets 2.1.0
    Uninstalling tensorflow-datasets-2.1.0:
      Successfully uninstalled tensorflow-datasets-2.1.0


In [4]:
pip install -q tensorflow>=2 tfds-nightly matplotlib

In [5]:
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds 
import pandas as pd

In [6]:
# Loading the wikipedia dataset.
DATASET_NAME = 'wikipedia/20190301.en'
# DATASET_NAME = 'wikipedia/20190301.uk'

dataset, dataset_info = tfds.load(
    name=DATASET_NAME,
    data_dir='tmp',
    with_info=True,
    split=tfds.Split.TRAIN,
)

In [7]:
print(dataset_info)

tfds.core.DatasetInfo(
    name='wikipedia',
    version=1.0.0,
    description='Wikipedia dataset containing cleaned articles of all languages.
The datasets are built from the Wikipedia dump
(https://dumps.wikimedia.org/) with one split per language. Each example
contains the content of one full Wikipedia article with cleaning to strip
markdown and unwanted sections (references, etc.).',
    homepage='https://dumps.wikimedia.org',
    features=FeaturesDict({
        'text': Text(shape=(), dtype=tf.string),
        'title': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=5824596,
    splits={
        'train': 5824596,
    },
    supervised_keys=None,
    citation="""@ONLINE {wikidump,
        author = "Wikimedia Foundation",
        title  = "Wikimedia Downloads",
        url    = "https://dumps.wikimedia.org"
    }""",
    redistribution_info=license: "This work is licensed under the Creative Commons Attribution-ShareAlike 3.0 Unported License. To view a copy of this l

In [8]:
print(dataset)

<PrefetchDataset shapes: {text: (), title: ()}, types: {text: tf.string, title: tf.string}>


In [9]:
#Find how many train examples we have available
TRAIN_NUM_EXAMPLES = dataset_info.splits['train'].num_examples
print('Total number of articles: ', TRAIN_NUM_EXAMPLES)

Total number of articles:  5824596


In [10]:
#Print first article to view as an example
print('First article','\n======\n')
for example in dataset.take(2):
    print('Title:','\n------')
    print(example['title'].numpy().decode('utf-8'))
    print()

    print('Text:', '\n------')
    print(example['text'].numpy().decode('utf-8'))

First article 

Title: 
------
Joseph Greenberg

Text: 
------
Joseph Harold Greenberg (May 28, 1915 – May 7, 2001) was an American linguist, known mainly for his work concerning linguistic typology and the genetic classification of languages.

Life

Early life and education 
(Main source: Croft 2003)

Joseph Greenberg was born on May 28, 1915 to Jewish parents in Brooklyn, New York. His first great interest was music. At the age of 14, he gave a piano concert in Steinway Hall. He continued to play the piano frequently throughout his life.

After finishing high school, he decided to pursue a scholarly career rather than a musical one. He enrolled at Columbia University in New York. During his senior year, he attended a class taught by Franz Boas concerning American Indian languages. With references from Boas and Ruth Benedict, he was accepted as a graduate student by Melville J. Herskovits at Northwestern University in Chicago. During the course of his graduate studies, Greenberg did f

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [75]:
sentences_list = []
word_list = []
stop_words = set(stopwords.words("english"))
punctuations = [".", ",", "!", "?", ";", "#", ":", "(", ")", "-", "–" "/", "http", "https", "www", "\\", "@", "&"]

for article in dataset.take(10):
    sentences = nltk.sent_tokenize(article['text'].numpy().decode('utf-8'))
    sentences_list.append(sentences)
    for sentence in sentences:
      words = nltk.word_tokenize(sentence)
      without_stop_words = [
      word for word in words if word not in stop_words]
      clean_words = [
                     word for word in without_stop_words if word not in punctuations]
      #word for word in words if word not in stop_words or any(punc in word for punc in punctuations)]

      word_list.append(clean_words)

In [76]:
word_list

[['Joseph',
  'Harold',
  'Greenberg',
  'May',
  '28',
  '1915',
  '–',
  'May',
  '7',
  '2001',
  'American',
  'linguist',
  'known',
  'mainly',
  'work',
  'concerning',
  'linguistic',
  'typology',
  'genetic',
  'classification',
  'languages'],
 ['Life',
  'Early',
  'life',
  'education',
  'Main',
  'source',
  'Croft',
  '2003',
  'Joseph',
  'Greenberg',
  'born',
  'May',
  '28',
  '1915',
  'Jewish',
  'parents',
  'Brooklyn',
  'New',
  'York'],
 ['His', 'first', 'great', 'interest', 'music'],
 ['At', 'age', '14', 'gave', 'piano', 'concert', 'Steinway', 'Hall'],
 ['He', 'continued', 'play', 'piano', 'frequently', 'throughout', 'life'],
 ['After',
  'finishing',
  'high',
  'school',
  'decided',
  'pursue',
  'scholarly',
  'career',
  'rather',
  'musical',
  'one'],
 ['He', 'enrolled', 'Columbia', 'University', 'New', 'York'],
 ['During',
  'senior',
  'year',
  'attended',
  'class',
  'taught',
  'Franz',
  'Boas',
  'concerning',
  'American',
  'Indian',
  'langu

IT WORKS UNTIL THIS PART

In [79]:
# # Start a SparkSession

# # Start a SparkSession

# # Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("word2vec").getOrCreate()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-ba

In [81]:
# # Note this is an RDD approach, not dataframe. API is slightly different for dataframe
from pyspark.mllib.feature import Word2Vec
word2vec = Word2Vec()

In [82]:
# # Train the model
model = word2vec.fit(clean_words)

TypeError: ignored

In [ ]:
# synonyms = model.findSynonyms('restaurant', 5)

# for word, cosine_distance in synonyms:
#     print("{}: {}".format(word, cosine_distance))

In [ ]:
# def cosine_similarity(x, y):
#     return x.dot(y)/(x.norm(2)*y.norm(2))

In [ ]:
# x = model.transform('location')
# y = model.transform('restaurant')
# cosine_similarity(x, y)

In [ ]:
# synonyms = model.findSynonyms('diner', 2)

# for word, cosine_distance in synonyms:
#     print("{}: {}".format(word, cosine_distance))

In [ ]:
# def cosine_similarity(x, y):
#     return x.dot(y)/(x.norm(2)*y.norm(2))

# x = model.transform('diner')
# y = model.transform('slice,')
# cosine_similarity(x, y)